In [1]:
import pandas as pd
import numpy as np
from jqdatasdk import *
import grequests
import requests
import json

auth('13810650842','123456a')
pd.options.display.max_rows=None
pd.options.display.max_columns=None

D:\develop\python\Anaconda3\lib\site-packages\grequests.py:21: MonkeyPatchWarning: Monkey-patching ssl after ssl has already been imported may lead to errors, including RecursionError on Python 3.6. It may also silently lead to incorrect behaviour on Python 3.7. Please monkey-patch earlier. See https://github.com/gevent/gevent/issues/1016. Modules that had direct imports (NOT patched): ['urllib3.util (D:\\develop\\python\\Anaconda3\\lib\\site-packages\\urllib3\\util\\__init__.py)', 'thriftpy2.transport._ssl (D:\\develop\\python\\Anaconda3\\lib\\site-packages\\thriftpy2\\transport\\_ssl.py)', 'urllib3.contrib.pyopenssl (D:\\develop\\python\\Anaconda3\\lib\\site-packages\\urllib3\\contrib\\pyopenssl.py)']. 
  curious_george.patch_all(thread=False, select=False)


auth success 


D:\develop\python\Anaconda3\lib\site-packages\gevent\hub.py:154: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [12]:
df_r = pd.read_excel('all.xlsx', index_col=0, dtype={'code':str})
df_r = df_r[~df_r.account.isin(['华泰','华宝'])]

df_nav = pd.read_excel('nav.xlsx', index_col=0, dtype={'code':str})
df_nav.date = df_nav.date.astype(np.datetime64)

In [13]:

def calc_date(x):
    op = x['deal_type']
    if op == '分红':
        # 日期游标（除权日，可能在给定日期前后的一个范围内）
        # 非货币基金的分红，通常稍有一个月触发 2 次及以上的
        date_cursor = x['date']
        # 上边界，15 天以前
        before = pd.DatetimeIndex(end=date_cursor, freq='D', periods=15)[0]
        # 下边界 15 天以后
        after = pd.DatetimeIndex(start=date_cursor, freq='D', periods=15)[-1]
        
        df_temp = df_nav[(df_nav['code'] == x['code']) & (df_nav['date'] >= before) & (df_nav['date'] <= after) & (df_nav['deal_type'] == '分红')]
#         print(date_cursor, before, after, df_temp)
        if len(df_temp) > 0:
            return str(df_temp.date.values[0])[0:10]
        else:
            return '无法补充'
        pass
    else:
        return x['date']
    pass

def calc_nav_unit(x):
    df_temp = df_nav[(df_nav['code'] == x['code']) & (df_nav['date'] == x['date'])]
#     print(df_temp)
    if len(df_temp) > 0:
        return df_temp.nav_unit.values[0]
    else:
#         return '无法补充'
        return x['nav_unit']

def calc_nav_acc(x):
    df_temp = df_nav[(df_nav['code'] == x['code']) & (df_nav['date'] == x['date'])]
#     print(df_temp)
    if len(df_temp) > 0:
        return df_temp.nav_acc.values[0]
    else:
#         return '无法补充'
        return x['nav_acc']

df = df_r
df['date'] = df.apply(calc_date, axis=1)
df['nav_unit'] = df.apply(calc_nav_unit, axis=1)
df['nav_acc'] = df.apply(calc_nav_acc, axis=1)
# df['acc_比较'] = df['nav_acc'] == df['累计净值']
# df = df[['code','name', 'deal_type', 'date', '补充日期','nav_unit','单位净值','nav_比较', 'nav_acc','累计净值', "account"]]
# df[df['nav_比较'] == False]
df

D:\develop\python\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Creating a DatetimeIndex by passing range endpoints is deprecated.  Use `pandas.date_range` instead.
  
D:\develop\python\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: Creating a DatetimeIndex by passing range endpoints is deprecated.  Use `pandas.date_range` instead.
  # Remove the CWD from sys.path while we load stuff.


,id,date,time,code,name,deal_type,nav_unit,nav_acc,volume,deal_money,fee,occur_money,account,category1,category2,category3,category_id,unique_id,note
57,58,2016-05-11,14:59:00,161725,招商中证白酒指数分级,买入,0.8820,0.8970,566.33,499.50,0.50,500.00,康力泉_天天,A 股,行业股,全指消费,148,201605129898160512303000022391,无
58,59,2016-05-13,14:59:00,000179,广发美国房地产指数,买入,1.2420,1.4020,402.05,499.35,0.65,500.00,康力泉_天天,海外成熟,美国,房地产,314,20160517273030009703781,无
61,62,2016-05-24,14:59:00,000179,广发美国房地产指数,卖出,1.2270,1.3870,-402.05,490.85,2.47,488.38,康力泉_天天,海外成熟,美国,房地产,314,20160526273030009796140,无
62,63,2016-05-27,14:59:00,161725,招商中证白酒指数分级,卖出,0.8440,0.8590,-566.33,475.59,2.39,473.20,康力泉_天天,A 股,行业股,全指消费,148,201605309898160530303000020415,无
90,91,2016-09-14,14:59:00,000614,华安德国30(DAX)联接,买入,1.0060,1.0060,3375.67,3395.92,4.08,3400.00,康力泉_天天,海外成熟,德国,德国30,321,20160920040303185335626,无
109,110,2017-01-13,14:59:00,001180,广发医药卫生联接A,买入,0.7815,0.7815,4728.82,3695.57,4.43,3700.00,康力泉_天天,A 股,行业股,全指医药,142,20170116273030010796153,无
110,111,2017-01-13,14:59:00,001180,广发医药卫生联接A,买入,0.7815,0.7815,4728.82,3695.57,4.43,3700.00,康力泉_天天,A 股,行业股,全指医药,142,20170116273030010796158,无
113,114,2017-01-25,14:59:00,100032,富国中证红利指数增强,买入,1.1070,2.4920,3571.88,3954.07,5.93,3960.00,康力泉_天天,A 股,红利价值,中证红利,131,20170126103030003315532,无
114,115,2017-01-25,14:59:00,000968,广发中证养老,买入,0.9595,0.9595,4122.20,3955.25,4.75,3960.00,康力泉_天天,A 股,行业股,养老产业,141,20170126273030010860386,无
120,121,2017-02-15,14:59:00,003376,广发中债7-10年国开债指数A,买入,0.9504,0.9504,4164.58,3958.02,1.98,3960.00,康力泉_天天,债券,国内债券,纯债,511,20170216273030010930699,无
